In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/22/cronograma/Major-League-Soccer-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text(strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

C:\Users\eduar\AppData\Local\Temp\ipykernel_34040\3601621580.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [3]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None

# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

tabela[['Data','Em casa','Visitante','Link Relatório']].tail(10)

,Data,Em casa,Visitante,Link Relatório
551,2024-11-03,Houston Dynamo,Seattle Sounders FC,/pt/partidas/ebd78237/Houston-Dynamo-Seattle-S...
552,2024-11-02,Minnesota Utd,Real Salt Lake,/pt/partidas/3e2d4a20/Minnesota-United-Real-Sa...
553,2024-11-03,Vancouver W'caps,LAFC,/pt/partidas/9e726acf/Vancouver-Whitecaps-FC-L...
554,2024-11-09,FC Cincinnati,NYCFC,/pt/partidas/0761b1cf/FC-Cincinnati-New-York-C...
555,2024-11-09,Orlando City,Charlotte,/pt/partidas/0b0ad266/Orlando-City-Charlotte-F...
556,2024-11-08,LAFC,Vancouver W'caps,/pt/partidas/be9f2b77/Los-Angeles-FC-Vancouver...
557,2024-11-09,Inter Miami,Atlanta Utd,/pt/partidas/5c5b1c24/Inter-Miami-Atlanta-Unit...
558,2024-11-23,NYCFC,NY Red Bulls,/pt/partidas/7779d63d/New-York-City-FC-New-Yor...
559,2024-11-24,LA Galaxy,Minnesota Utd,/pt/partidas/37d92c84/LA-Galaxy-Minnesota-Unit...
560,2024-11-24,Orlando City,Atlanta Utd,/pt/partidas/1b1e9591/Orlando-City-Atlanta-Uni...


In [4]:
tabela.to_csv(r'C:\Users\eduar\OneDrive\Desktop\DEV\Football\MLS\links.csv', index= False)